In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
import numpy as np
import math
import random

In [4]:
import time

In [5]:
def show_state(env, episode,step, info=""):
    plt.figure(99999,figsize=[8,6])
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("episode: {} step: {} ".format(episode,step))
    #plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [6]:
import gym

In [7]:
use_cuda = torch.cuda.is_available()
device=torch.device("cuda:4" if use_cuda else "cpu")

In [8]:
###### PARAMS ######
learning_rate = 0.01
num_episodes = 1000
gamma=0.99
#gamma=0.85
egreedy = 0.9
egreedy_final = 0.02
egreedy_decay = 500

report_interval=10

score_to_solve = 195

hidden_layer_size=64

replay_memory_size=50000

In [9]:
env = gym.make('CartPole-v1')

In [10]:
#env = gym.wrappers.Monitor(env, '../mp4/sandbox10',video_callable=lambda episode_id: True,force=True)
#env = gym.wrappers.Monitor(env, '../mp4/cartpole-5',video_callable=lambda episode_id: episode_id%10==0)

In [11]:
number_of_inputs=env.observation_space.shape[0]
number_of_outputs=env.action_space.n

In [12]:
def calculate_epsilon(steps_done):
    epsilon = egreedy_final + (egreedy - egreedy_final) * \
              math.exp(-1. * steps_done / egreedy_decay )
    return epsilon

In [ ]:
class ExperienceReplay():
    def __init__(self, capacity):
        self.capacity=capacity
        self.memory=[]
        self.position=0
        
    
    def push(self, state,
             action, new_state,
             reward, done):
        pass
    
    def sample(self):
        pass
    
    def __len__(self):
        pass
    

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs,hidden_layer_size)
        self.linear2 = nn.Linear(hidden_layer_size,number_of_outputs)
        
        self.activation=nn.Tanh()
        
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)
        
        return output2

In [14]:
class QNet_Agent():
    def __init__(self):
        self.nn = NeuralNetwork().to(device)
        self.loss_function = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        
    def select_action(self,state,epsilon):
        
        random_for_egreedy=torch.rand(1).item()
        
        if random_for_egreedy>epsilon:
            self.nn.eval()
            with torch.no_grad():
                state=torch.Tensor(state).to(device)
                predicted_value_from_nn=self.nn(state)
                action=torch.argmax(predicted_value_from_nn).item()
        else:
            action=env.action_space.sample()
                
                
        return action
    
    def optimize(self, state, action, new_state, reward, done):
        
        state=torch.Tensor(state).to(device)
        new_state=torch.Tensor(new_state).to(device)
        reward=torch.Tensor([reward]).to(device)
        
        
        if done:
            target_value=reward
        else:
            self.nn.eval()
            new_state_values=self.nn(new_state).detach()
            max_new_state_values=torch.max(new_state_values)
            target_value=reward + gamma*max_new_state_values
        
        self.nn.train()
        #the view call needed to make it a tensor
        predicted_value=self.nn(state)[action].view(-1)
        
        loss=self.loss_function(predicted_value,target_value)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        

In [ ]:
memory=ExperienceReplay(replay_memory_size)

In [15]:
qnet_agent=QNet_Agent()

In [16]:
seed_value=23
env.seed(seed_value)
torch.manual_seed(seed_value)
random.seed(seed_value)

In [ ]:
%%time

num_episodes=1000

steps_total=np.full([num_episodes],-999,dtype=np.int32)

frames_total=0

solved_after = 0
solved = False

start_time = time.time()

for i_episode in range(num_episodes):
    
    state = env.reset()
    #for step in range(100):
    step=0
    while True:
        
        step+=1
        frames_total += 1
        
        epsilon=calculate_epsilon(frames_total)
        
        #action=env.action_space.sample()
        action=qnet_agent.select_action(state,epsilon)
        
        new_state, reward, done, info = env.step(action)
        memeory.push(state, action, new_state,
                     reward, done)
        
        qnet_agent.optimize()
        
        state=new_state
        
        if done:
            steps_total[i_episode]=step
            
            if i_episode>100:
                mean_reward_100 = np.sum(steps_total[i_episode-100:i_episode])/100
            
                if (mean_reward_100 > score_to_solve and solved == False):
                    print("SOLVED! After %i episodes " % i_episode)
                    solved_after = i_episode
                    solved = True
            
            if (i_episode % report_interval == 0 and i_episode>1):
                print("**** Episode  {} **** ".format(i_episode))
                recent_avg_reward=np.average(steps_total[i_episode-report_interval:i_episode])
                print("Recent average reward: {}".format(recent_avg_reward))
                if i_episode>100:
                    print("Reward over last 100: {}".format(mean_reward_100))
                full_avg_so_far=np.average(steps_total[:i_episode])
                print("Average over all episodes so far: {}".format(full_avg_so_far))
                print("epsilon: {}".format(epsilon))
            
                #print("Episode {} finished after: {}".format(i_episode,step))
            break
            
if solved:
    print("Solved after %i episodes" % solved_after)
        

**** Episode  10 **** 
Recent average reward: 23.9
Average over all episodes so far: 23.9
epsilon: 0.5537469805471175
**** Episode  20 **** 
Recent average reward: 14.0
Average over all episodes so far: 18.95
epsilon: 0.4185854512433708
**** Episode  30 **** 
Recent average reward: 41.6
Average over all episodes so far: 26.5
epsilon: 0.17139948016428447
**** Episode  40 **** 
Recent average reward: 42.9
Average over all episodes so far: 30.6
epsilon: 0.09369403852113245
**** Episode  50 **** 
Recent average reward: 39.8
Average over all episodes so far: 32.44
epsilon: 0.05194200476696485
**** Episode  60 **** 
Recent average reward: 58.6
Average over all episodes so far: 36.8
epsilon: 0.029874166552438394
**** Episode  70 **** 
Recent average reward: 62.9
Average over all episodes so far: 40.52857142857143
epsilon: 0.022669582004333038
**** Episode  80 **** 
Recent average reward: 119.6
Average over all episodes so far: 50.4125
epsilon: 0.02025459579921895
**** Episode  90 **** 
Recent

**** Episode  590 **** 
Recent average reward: 246.9
Reward over last 100: 411.65
Average over all episodes so far: 281.6101694915254
epsilon: 0.02
**** Episode  600 **** 
Recent average reward: 313.2
Reward over last 100: 397.75
Average over all episodes so far: 282.13666666666666
epsilon: 0.02
**** Episode  610 **** 
Recent average reward: 483.5
Reward over last 100: 396.1
Average over all episodes so far: 285.4377049180328
epsilon: 0.02
**** Episode  620 **** 
Recent average reward: 396.2
Reward over last 100: 390.66
Average over all episodes so far: 287.2241935483871
epsilon: 0.02
**** Episode  630 **** 
Recent average reward: 61.8
Reward over last 100: 346.84
Average over all episodes so far: 283.6460317460317
epsilon: 0.02


In [ ]:
plt.plot(steps_total)
plt.show()

In [ ]:
print("Average number of steps: {}". format(np.average(steps_total)))

In [ ]:
print("Average number of steps=reward: {}". format(np.average(steps_total)))
print("Average number of steps=reward in last 100 episodes: {}". format(np.average(steps_total[-100:])))



plt.figure(1,figsize=[12,5])
plt.title("Rewards")
plt.bar(torch.arange(len(rewards_total)), rewards_total,alpha=0.6, color='green')
#plt.plot(rewards_total)
plt.show()

In [ ]:
plt.figure(2,figsize=[12,5])
plt.title("Steps to finish episode")
plt.plot(steps_total)
plt.show()